In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [0]:
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 15
USE_CUDA = torch.cuda.is_available()

In [0]:
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [4]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 10.6MB/s]


In [0]:
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

In [0]:
trainset, valset = trainset.split(split_ratio = 0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset),
                                                             batch_size=BATCH_SIZE, shuffle=True, repeat=False)

In [7]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('[TRAIN] : %d \t [VALID] : %d \t [TEST] : %d \t [VOCAB] %d \t [CLASSES] %d'%(
    len(trainset), len(valset), len(testset), vocab_size, n_classes
))

[TRAIN] : 20000 	 [VALID] : 5000 	 [TEST] : 25000 	 [VOCAB] 46159 	 [CLASSES] 2


In [0]:
class BasicRNN(nn.Module):
  def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
    super(BasicRNN, self).__init__()
    print('Building RNN')
    self.n_layers = n_layers
    self.embed = nn.Embedding(n_vocab, embed_dim)
    self.hidden_dim = hidden_dim
    self.dropout = nn.Dropout(dropout_p)
    self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first=True)
    self.out = nn.Linear(self.hidden_dim, n_classes)
  
  def forward(self, x):
    x = self.embed(x)
    h_0 = self._init_state(batch_size = x.size(0))
    x, _ = self.rnn(x, h_0)
    h_t = x[:, -1, :]
    self.dropout(h_t)
    logit = torch.sigmoid(self.out(h_t))
    return logit

  def _init_state(self, batch_size = 1):
    weight = next(self.parameters()).data
    return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()


In [0]:
def train(model, optimizer, train_iter):
  model.train()
  for b, batch in enumerate(train_iter):
    x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
    y.data.sub_(1)
    optimizer.zero_grad()

    logit = model(x)
    loss = F.cross_entropy(logit, y)
    loss.backward()
    optimizer.step()

    if b % 50 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
          e, b * len(x), len(train_iter.dataset), 100. * b / len(train_iter), loss.item()
      ))

In [0]:
def evaluate(model, val_iter):
  model.eval()
  corrects, total_loss = 0, 0

  for batch in val_iter:
    x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
    y.data.sub_(1)
    logit = model(x)
    loss = F.cross_entropy(logit, y, reduction='sum')
    total_loss += loss.item()
    corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()

  size = len(val_iter.dataset)
  avg_loss = total_loss / size
  avg_accuracy = 100.0 * corrects / size
  return avg_loss, avg_accuracy

In [21]:
model = BasicRNN(
    n_layers=1, hidden_dim = 256, n_vocab = vocab_size, embed_dim = 128, n_classes=n_classes, dropout_p=0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
for e in range(1, EPOCHS + 1):
  train(model, optimizer, train_iter)
  val_loss, val_accuracy = evaluate(model, val_iter)
  print('[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f'%(e, val_loss, val_accuracy))

Building RNN
Train Epoch: 1 [0/20000 (0%)]	Loss:0.693788
Train Epoch: 1 [5000/20000 (25%)]	Loss:0.691996
Train Epoch: 1 [10000/20000 (50%)]	Loss:0.693209
Train Epoch: 1 [15000/20000 (75%)]	Loss:0.693421
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy: 49.24
Train Epoch: 2 [0/20000 (0%)]	Loss:0.693119
Train Epoch: 2 [5000/20000 (25%)]	Loss:0.692230
Train Epoch: 2 [10000/20000 (50%)]	Loss:0.690141
Train Epoch: 2 [15000/20000 (75%)]	Loss:0.692805
[EPOCH: 2], Validation Loss:  0.70 | Validation Accuracy: 49.22
Train Epoch: 3 [0/20000 (0%)]	Loss:0.682445
Train Epoch: 3 [5000/20000 (25%)]	Loss:0.689822
Train Epoch: 3 [10000/20000 (50%)]	Loss:0.693898
Train Epoch: 3 [15000/20000 (75%)]	Loss:0.693248
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy: 49.88
Train Epoch: 4 [0/20000 (0%)]	Loss:0.693290
Train Epoch: 4 [5000/20000 (25%)]	Loss:0.693272
Train Epoch: 4 [10000/20000 (50%)]	Loss:0.692922
Train Epoch: 4 [15000/20000 (75%)]	Loss:0.693106
[EPOCH: 4], Validation Loss:  0.69 

In [0]:
test_loss, test_acc = evaluate(model, test_iter)
print('Test Loss:%5.2f | Test Accuracy: %5.2f'%(test_loss, test_acc))

In [0]:
class BasicGRU(nn.Module):
  def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
    super(BasicGRU, self).__init__()
    print('Building GRU')
    self.n_layers = n_layers
    self.embed = nn.Embedding(n_vocab, embed_dim)
    self.hidden_dim = hidden_dim
    self.dropout = nn.Dropout(dropout_p)
    self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
    self.out = nn.Linear(self.hidden_dim, n_classes)

  def forward(self, x):
    x = self.embed(x)
    h_0 = self._init_state(batch_size = x.size(0))
    x, _ = self.gru(x, h_0)
    h_t = x[:, -1, :]
    self.dropout(h_t)
    logit = torch.sigmoid(self.out(h_t))
    return logit

  def _init_state(self, batch_size = 1):
    weight = next(self.parameters()).data
    return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [26]:
model = BasicGRU(
    n_layers=1, hidden_dim=256, n_vocab=vocab_size, embed_dim=128, n_classes=n_classes, dropout_p=0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
for e in range(1, EPOCHS + 1):
  train(model, optimizer, train_iter)
  val_loss, val_accuracy = evaluate(model, val_iter)
  print('[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f' %(e, val_loss, val_accuracy))

Building GRU
Train Epoch: 1 [0/20000 (0%)]	Loss:0.698118
Train Epoch: 1 [5000/20000 (25%)]	Loss:0.694385
Train Epoch: 1 [10000/20000 (50%)]	Loss:0.693067
Train Epoch: 1 [15000/20000 (75%)]	Loss:0.693173
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy: 50.48
Train Epoch: 2 [0/20000 (0%)]	Loss:0.692846
Train Epoch: 2 [5000/20000 (25%)]	Loss:0.693092
Train Epoch: 2 [10000/20000 (50%)]	Loss:0.693500
Train Epoch: 2 [15000/20000 (75%)]	Loss:0.692800
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy: 51.48
Train Epoch: 3 [0/20000 (0%)]	Loss:0.693493
Train Epoch: 3 [5000/20000 (25%)]	Loss:0.692524
Train Epoch: 3 [10000/20000 (50%)]	Loss:0.694783
Train Epoch: 3 [15000/20000 (75%)]	Loss:0.693296
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy: 49.96
Train Epoch: 4 [0/20000 (0%)]	Loss:0.693011
Train Epoch: 4 [5000/20000 (25%)]	Loss:0.693126
Train Epoch: 4 [10000/20000 (50%)]	Loss:0.693203
Train Epoch: 4 [15000/20000 (75%)]	Loss:0.693081
[EPOCH: 4], Validation Loss:  0.69 

In [28]:
test_loss, test_acc = evaluate(model, test_iter)
print('Test Loss: %5.2f | Test Accuracy: %5.2f' %(test_loss, test_acc))

Test Loss:  0.49 | Test Accuracy: 81.78


In [0]:
class BasicLSTM(nn.Module):
  def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
    super(BasicLSTM, self).__init__()
    print('Building LSTM')
    self.n_layers = n_layers
    self.embed = nn.Embedding(n_vocab, embed_dim)
    self.hidden_dim = hidden_dim
    self.dropout = nn.Dropout(dropout_p)
    self.lstm = nn.LSTM(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)
    self.out = nn.Linear(self.hidden_dim, n_classes)
  
  def forward(self, x):
    x = self.embed(x)
    h_0 = self._init_state(batch_size = x.size(0))
    c_0 = self._init_state(batch_size = x.size(0))

    x, _ = self.lstm(x, (h_0, c_0))
    h_t = x[:, -1, :]
    self.dropout(h_t)
    logit = torch.sigmoid(self.out(h_t))
    return logit

  def _init_state(self, batch_size=1):
    weight = next(self.parameters()).data
    return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [30]:
model = BasicLSTM(
    n_layers=1, hidden_dim=256, n_vocab=vocab_size, embed_dim=128, n_classes=n_classes, dropout_p=0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
for e in range(1, EPOCHS + 1):
  train(model, optimizer, train_iter)
  val_loss, val_accuracy = evaluate(model, val_iter)
  print('[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f' %(e, val_loss, val_accuracy))

Building LSTM
Train Epoch: 1 [0/20000 (0%)]	Loss:0.693781
Train Epoch: 1 [5000/20000 (25%)]	Loss:0.693246
Train Epoch: 1 [10000/20000 (50%)]	Loss:0.690423
Train Epoch: 1 [15000/20000 (75%)]	Loss:0.693761
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy: 48.14
Train Epoch: 2 [0/20000 (0%)]	Loss:0.694339
Train Epoch: 2 [5000/20000 (25%)]	Loss:0.691478
Train Epoch: 2 [10000/20000 (50%)]	Loss:0.691857
Train Epoch: 2 [15000/20000 (75%)]	Loss:0.691453
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy: 50.16
Train Epoch: 3 [0/20000 (0%)]	Loss:0.698122
Train Epoch: 3 [5000/20000 (25%)]	Loss:0.694497
Train Epoch: 3 [10000/20000 (50%)]	Loss:0.690712
Train Epoch: 3 [15000/20000 (75%)]	Loss:0.691720
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy: 49.96
Train Epoch: 4 [0/20000 (0%)]	Loss:0.692962
Train Epoch: 4 [5000/20000 (25%)]	Loss:0.691455
Train Epoch: 4 [10000/20000 (50%)]	Loss:0.695489
Train Epoch: 4 [15000/20000 (75%)]	Loss:0.693069
[EPOCH: 4], Validation Loss:  0.69

In [31]:
test_loss, test_acc = evaluate(model, test_iter)
print('Test Loss: %5.2f | Test Accuracy: %5.2f'%(test_loss, test_acc))

Test Loss:  0.56 | Test Accuracy: 77.00
